# MCP Client Example

Here is the example of how you can use Kotlin MCP client.

Let's start with adding dependencies and imports.

We'll need:
- Kotlin SDK Client
- Ktor client
- Kotlinx.coroutines

In [10]:
%use coroutines (1.10.2)
%use ktor-client (0.3.0-5)

USE {
    dependencies(
        "io.modelcontextprotocol:kotlin-sdk-client-jvm:0.7.7",
    )
}

In [11]:
import kotlinx.coroutines.runBlocking
import io.modelcontextprotocol.kotlin.sdk.EmptyJsonObject
import io.modelcontextprotocol.kotlin.sdk.Implementation
import io.modelcontextprotocol.kotlin.sdk.TextContent
import io.modelcontextprotocol.kotlin.sdk.client.Client
import io.modelcontextprotocol.kotlin.sdk.client.ClientOptions
import io.modelcontextprotocol.kotlin.sdk.client.StdioClientTransport
import io.modelcontextprotocol.kotlin.sdk.client.StreamableHttpClientTransport
import io.ktor.client.HttpClient
import io.ktor.client.call.body
import io.ktor.client.request.get
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.JsonObject
import io.ktor.client.plugins.logging.*
import io.ktor.client.plugins.sse.SSE

First, let's create the Ktor HTTP Client first.
We are installing Server-Sent Events (SSE) plugin, and, also, a Logging plugin for convenience.

In [12]:
val httpClient = HttpClient {
    install(SSE)
    install(Logging) {
        level = LogLevel.ALL
        logger = object : Logger {
            override fun log(message: String) {
                DISPLAY(HTML("<details><summary>HTTP Log</summary><pre>$message</pre></details>"))
            }
        }
    }
}


Now let's create an MCP client:

In [13]:
val mcpClient = Client(
    clientInfo = Implementation(
        name = "my-client",
        version = "1.0.0"
    ),
    options = ClientOptions(),
)

Now we can connect an MCP client to remote MCP server via StreamableHttpClientTransport

In [14]:
runBlocking {
    mcpClient.connect(
        StreamableHttpClientTransport(
            client = httpClient,
            url = "https://learn.microsoft.com/api/mcp"
        )
    )

    println("✅ Server version: ${mcpClient.serverVersion}")
    println("✅ Server capabilities: ${mcpClient.serverCapabilities}")
}

HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: POST
COMMON HEADERS
-> Accept: application/json, text/event-stream
-> Accept-Charset: UTF-8
CONTENT HEADERS
-> Content-Length: 200
-> Content-Type: application/json
BODY Content-Type: application/json
BODY START
{"id":5,"method":"initialize","params":{"protocolVersion":"2025-03-26","capabilities":{"experimental":{},"sampling":{}},"clientInfo":{"name":"my-client","version":"1.0.0"},"_meta":{}},"jsonrpc":"2.0"}
BODY END

HTTP Log RESPONSE: 200 OK
METHOD: POST
FROM: https://learn.microsoft.com/api/mcp
COMMON HEADERS
-> Akamai-Cache-Status: Miss from child, Miss from parent
-> Cache-Control: no-cache, no-store
-> Connection: keep-alive
-> Content-Encoding: identity
-> Content-Length: 1914
-> Content-Type: text/event-stream
-> Date: Fri, 14 Nov 2025 14:12:48 GMT
-> Expires: Fri, 14 Nov 2025 14:12:48 GMT
-> Mcp-Session-Id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
-> Request-Context: appId=cid-v1:ccd19013-8f20-4877-8d3f-84a0107e7119
-> Set-Cookie: ASLBSA=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; Path=/; Secure; HttpOnly;; ASLBSACORS=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; SameSite=none; Path=/; Secure; HttpOnly;
-> Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
-> X-Content-Type-Options: nosniff
-> X-Powered-By: ASP.NET
-> nel: {"report_to":"network-errors","max_age":604800,"success_fraction":0.01,"failure_fraction":1.0}
-> report-to: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://mdec.nelreports.net/api/report?cat=mdocs"}]}
-> x-azure-ref: 20251114T141248Z-169b4b8d65848xdthC1AMS8x840000000xfg00000000arwf
BODY Content-Type: text/event-stream
BODY START
event: message
data: {"result":{"protocolVersion":"2025-03-26","capabilities":{"logging":{},"prompts":{},"resources":{},"tools":{"listChanged":true}},"serverInfo":{"name":"Microsoft Learn MCP Server","version":"1.0.0"},"instructions":"# Microsoft Learn MCP Server\r\n\r\nThis server gives structured access to official Microsoft and Azure documentation via three tools:\r\n\r\n## Tools\r\n\r\n### microsoft_docs_search\r\nSearch official documentation and return up to 10 concise, high-quality content chunks (max 500 tokens each), including title, URL, and excerpt.\r\n\r\n- Use first to get a quick, reliable overview\r\n- Ideal for grounding answers in Microsoft knowledge\r\n\r\n### microsoft_code_sample_search\r\nSearch for code snippets and examples in official Microsoft Learn documentation and return up to 20 relevant, high-quality code samples.\r\n\r\n- Use when you need to provide sample Microsoft/Azure related code in your answers. \r\n- Ideal for generating code snippets or practical implementation examples. \r\n- Optional parameter \u0060language\u0060 can filter results. \r\n\r\n### microsoft_docs_fetch\r\nFetch and convert full Microsoft documentation pages to markdown.\r\n\r\n- Use after search when you need full content from a specific URL\r\n- Required for detailed tutorials, troubleshooting, prerequisites, code samples, or when search results are incomplete or outdated\r\n\r\n## Workflow\r\n\r\n1. Use \u0060microsoft_docs_search\u0060 to find relevant documents.\r\n2. If you need code examples or practical snippets, use \u0060microsoft_code_sample_search\u0060.\r\n3. If deeper or complete information is needed, use \u0060microsoft_docs_fetch\u0060.\r\n\r\n**Search gives breadth. Code Sample Search gives practical examples. Fetch gives depth.**\r\n\r\nAll content comes from Microsoft Learn or official sources, returned in clean markdown format."},"id":5,"jsonrpc":"2.0"}


BODY END

HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: POST
COMMON HEADERS
-> Accept: application/json, text/event-stream
-> Accept-Charset: UTF-8
-> mcp-session-id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
CONTENT HEADERS
-> Content-Length: 76
-> Content-Type: application/json
BODY Content-Type: application/json
BODY START
{"method":"notifications/initialized","params":{"_meta":{}},"jsonrpc":"2.0"}
BODY END

HTTP Log RESPONSE: 202 Accepted
METHOD: POST
FROM: https://learn.microsoft.com/api/mcp
COMMON HEADERS
-> Akamai-Cache-Status: Miss from child, Miss from parent
-> Cache-Control: no-cache, no-store
-> Connection: keep-alive
-> Content-Encoding: identity
-> Content-Length: 0
-> Date: Fri, 14 Nov 2025 14:12:48 GMT
-> Expires: Fri, 14 Nov 2025 14:12:48 GMT
-> Mcp-Session-Id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
-> Request-Context: appId=cid-v1:ccd19013-8f20-4877-8d3f-84a0107e7119
-> Set-Cookie: ASLBSA=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; Path=/; Secure; HttpOnly;; ASLBSACORS=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; SameSite=none; Path=/; Secure; HttpOnly;
-> Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
-> X-Content-Type-Options: nosniff
-> X-Powered-By: ASP.NET
-> nel: {"report_to":"network-errors","max_age":604800,"success_fraction":0.01,"failure_fraction":1.0}
-> report-to: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://mdec.nelreports.net/api/report?cat=mdocs"}]}
-> x-azure-ref: 20251114T141248Z-169b4b8d6588t6fthC1AMSxncg00000018v000000000ruuv
BODY Content-Type: null
BODY START

BODY END

HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: GET
COMMON HEADERS
-> Accept: application/json
-> Accept-Charset: UTF-8
-> mcp-session-id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
CONTENT HEADERS
-> Content-Length: 0
-> Accept: text/event-stream
-> Cache-Control: no-store
BODY Content-Type: null
BODY START

BODY END

✅ Server version: Implementation(name=Microsoft Learn MCP Server, version=1.0.0)
✅ Server capabilities: ServerCapabilities(experimental={}, sampling={}, logging={}, prompts=Prompts(listChanged=null), resources=Resources(subscribe=null, listChanged=null), tools=Tools(listChanged=true))


Now we can send ping request to MCP server:

In [15]:
runBlocking {
    mcpClient.ping()
}

HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: POST
COMMON HEADERS
-> Accept: application/json, text/event-stream
-> Accept-Charset: UTF-8
-> mcp-session-id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
CONTENT HEADERS
-> Content-Length: 52
-> Content-Type: application/json
BODY Content-Type: application/json
BODY START
{"id":6,"method":"ping","params":{},"jsonrpc":"2.0"}
BODY END

HTTP Log RESPONSE: 200 OK
METHOD: POST
FROM: https://learn.microsoft.com/api/mcp
COMMON HEADERS
-> Akamai-Cache-Status: Miss from child, Miss from parent
-> Cache-Control: no-cache, no-store
-> Connection: keep-alive
-> Content-Encoding: identity
-> Content-Length: 59
-> Content-Type: text/event-stream
-> Date: Fri, 14 Nov 2025 14:12:49 GMT
-> Expires: Fri, 14 Nov 2025 14:12:49 GMT
-> Mcp-Session-Id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
-> Request-Context: appId=cid-v1:ccd19013-8f20-4877-8d3f-84a0107e7119
-> Set-Cookie: ASLBSA=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; Path=/; Secure; HttpOnly;; ASLBSACORS=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; SameSite=none; Path=/; Secure; HttpOnly;
-> Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
-> X-Content-Type-Options: nosniff
-> X-Powered-By: ASP.NET
-> nel: {"report_to":"network-errors","max_age":604800,"success_fraction":0.01,"failure_fraction":1.0}
-> report-to: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://mdec.nelreports.net/api/report?cat=mdocs"}]}
-> x-azure-ref: 20251114T141249Z-169b4b8d6588pb5bhC1AMS2m0w00000017n000000000w3kt
BODY Content-Type: text/event-stream
BODY START
event: message
data: {"result":{},"id":6,"jsonrpc":"2.0"}


BODY END

EmptyRequestResult(_meta={})

Let's get a list of available tools:

In [16]:
runBlocking {
    val listToolsResult = mcpClient.listTools()

    for (tool in listToolsResult.tools) {
        DISPLAY(tool)
    }
}


HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: POST
COMMON HEADERS
-> Accept: application/json, text/event-stream
-> Accept-Charset: UTF-8
-> mcp-session-id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
CONTENT HEADERS
-> Content-Length: 68
-> Content-Type: application/json
BODY Content-Type: application/json
BODY START
{"id":7,"method":"tools/list","params":{"_meta":{}},"jsonrpc":"2.0"}
BODY END

HTTP Log RESPONSE: 200 OK
METHOD: POST
FROM: https://learn.microsoft.com/api/mcp
COMMON HEADERS
-> Akamai-Cache-Status: Miss from child, Miss from parent
-> Cache-Control: no-cache, no-store
-> Connection: keep-alive
-> Content-Encoding: identity
-> Content-Length: 4462
-> Content-Type: text/event-stream
-> Date: Fri, 14 Nov 2025 14:12:49 GMT
-> Expires: Fri, 14 Nov 2025 14:12:49 GMT
-> Mcp-Session-Id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
-> Request-Context: appId=cid-v1:ccd19013-8f20-4877-8d3f-84a0107e7119
-> Set-Cookie: ASLBSA=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; Path=/; Secure; HttpOnly;; ASLBSACORS=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; SameSite=none; Path=/; Secure; HttpOnly;
-> Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
-> X-Content-Type-Options: nosniff
-> X-Powered-By: ASP.NET
-> nel: {"report_to":"network-errors","max_age":604800,"success_fraction":0.01,"failure_fraction":1.0}
-> report-to: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://mdec.nelreports.net/api/report?cat=mdocs"}]}
-> x-azure-ref: 20251114T141249Z-169b4b8d6588t6fthC1AMSxncg00000018v000000000ruxq
BODY Content-Type: text/event-stream
BODY START
event: message
data: {"result":{"tools":[{"name":"microsoft_docs_search","title":"Microsoft Docs Search","description":"Search official Microsoft/Azure documentation to find the most relevant and trustworthy content for a user\u0027s query. This tool returns up to 10 high-quality content chunks (each max 500 tokens), extracted from Microsoft Learn and other official sources. Each result includes the article title, URL, and a self-contained content excerpt optimized for fast retrieval and reasoning. Always use this tool to quickly ground your answers in accurate, first-party Microsoft/Azure knowledge.\n\nThe \u0060question\u0060 parameter is no longer used, use \u0060query\u0060 instead.\n\n## Follow-up Pattern\nTo ensure completeness, use microsoft_docs_fetch when high-value pages are identified by search. The fetch tool complements search by providing the full detail. This is a required step for comprehensive results.","inputSchema":{"type":"object","properties":{"query":{"description":"a query or topic about Microsoft/Azure products, services, platforms, developer tools, frameworks, or APIs","type":"string","default":null},"question":{"description":"this parameter is no longer used, use query instead.","type":"string","default":null}}},"annotations":{"title":"Microsoft Docs Search","destructiveHint":false,"idempotentHint":true,"readOnlyHint":true}},{"name":"microsoft_code_sample_search","title":"Microsoft Code Sample Search","description":"Search for code snippets and examples in official Microsoft Learn documentation. This tool retrieves relevant code samples from Microsoft documentation pages providing developers with practical implementation examples and best practices for Microsoft/Azure products and services related coding tasks. This tool will help you use the **LATEST OFFICIAL** code snippets to empower coding capabilities.\n\n## When to Use This Tool\n- When you are going to provide sample Microsoft/Azure related code snippets in your answers.\n- When you are **generating any Microsoft/Azure related code**.\n\n## Usage Pattern\nInput a descriptive query, or SDK/class/method name to retrieve related code samples. The optional parameter \u0060language\u0060 can help to filter results.\n\nEligible values for \u0060language\u0060 parameter include: csharp javascript typescript python powershell azurecli al sql java kusto cpp go rust ruby php","inputSchema":{"type":"object","properties":{"query":{"description":"a descriptive query, SDK name, method name or code snippet related to Microsoft/Azure products, services, platforms, developer tools, frameworks, APIs or SDKs","type":"string"},"language":{"description":"Optio

Tool(name=microsoft_docs_search, title=Microsoft Docs Search, description=Search official Microsoft/Azure documentation to find the most relevant and trustworthy content for a user's query. This tool returns up to 10 high-quality content chunks (each max 500 tokens), extracted from Microsoft Learn and other official sources. Each result includes the article title, URL, and a self-contained content excerpt optimized for fast retrieval and reasoning. Always use this tool to quickly ground your answers in accurate, first-party Microsoft/Azure knowledge.

The `question` parameter is no longer used, use `query` instead.

## Follow-up Pattern
To ensure completeness, use microsoft_docs_fetch when high-value pages are identified by search. The fetch tool complements search by providing the full detail. This is a required step for comprehensive results., inputSchema=Input(properties={"query":{"description":"a query or topic about Microsoft/Azure products, services, platforms, developer tools, f

Tool(name=microsoft_code_sample_search, title=Microsoft Code Sample Search, description=Search for code snippets and examples in official Microsoft Learn documentation. This tool retrieves relevant code samples from Microsoft documentation pages providing developers with practical implementation examples and best practices for Microsoft/Azure products and services related coding tasks. This tool will help you use the **LATEST OFFICIAL** code snippets to empower coding capabilities.

## When to Use This Tool
- When you are going to provide sample Microsoft/Azure related code snippets in your answers.
- When you are **generating any Microsoft/Azure related code**.

## Usage Pattern
Input a descriptive query, or SDK/class/method name to retrieve related code samples. The optional parameter `language` can help to filter results.

Eligible values for `language` parameter include: csharp javascript typescript python powershell azurecli al sql java kusto cpp go rust ruby php, inputSchema=Inpu

Tool(name=microsoft_docs_fetch, title=Microsoft Docs Fetch, description=Fetch and convert a Microsoft Learn documentation page to markdown format. This tool retrieves the latest complete content of Microsoft documentation pages including Azure, .NET, Microsoft 365, and other Microsoft technologies.

## When to Use This Tool
- When search results provide incomplete information or truncated content
- When you need complete step-by-step procedures or tutorials
- When you need troubleshooting sections, prerequisites, or detailed explanations
- When search results reference a specific page that seems highly relevant
- For comprehensive guides that require full context

## Usage Pattern
Use this tool AFTER microsoft_docs_search when you identify specific high-value pages that need complete content. The search tool gives you an overview; this tool gives you the complete picture.

## URL Requirements
- The URL must be a valid link from the microsoft.com domain.

## Output Format
markdown with 

Let's call a tool:

In [17]:
runBlocking {
    val toolCallResult = mcpClient.callTool(
        name = "microsoft_docs_search",
        arguments = mapOf("query" to "Linux")
    )
    toolCallResult?.let {
        for (content in it.content) {
            println(content)
        }
    }
}


HTTP Log REQUEST: https://learn.microsoft.com/api/mcp
METHOD: POST
COMMON HEADERS
-> Accept: application/json, text/event-stream
-> Accept-Charset: UTF-8
-> mcp-session-id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
CONTENT HEADERS
-> Content-Length: 129
-> Content-Type: application/json
BODY Content-Type: application/json
BODY START
{"id":8,"method":"tools/call","params":{"name":"microsoft_docs_search","arguments":{"query":"Linux"},"_meta":{}},"jsonrpc":"2.0"}
BODY END

HTTP Log RESPONSE: 200 OK
METHOD: POST
FROM: https://learn.microsoft.com/api/mcp
COMMON HEADERS
-> Akamai-Cache-Status: Miss from child, Miss from parent
-> Cache-Control: no-cache, no-store
-> Connection: keep-alive
-> Content-Encoding: identity
-> Content-Length: 22458
-> Content-Type: text/event-stream
-> Date: Fri, 14 Nov 2025 14:12:50 GMT
-> Expires: Fri, 14 Nov 2025 14:12:50 GMT
-> Mcp-Session-Id: eyJjbGllbnRJbmZvIjp7Im5hbWUiOiJteS1jbGllbnQiLCJ0aXRsZSI6bnVsbCwidmVyc2lvbiI6IjEuMC4wIn0sInVzZXJJZENsYWltIjpudWxsfQ
-> Request-Context: appId=cid-v1:ccd19013-8f20-4877-8d3f-84a0107e7119
-> Set-Cookie: ASLBSA=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; Path=/; Secure; HttpOnly;; ASLBSACORS=0003562b174eb06eed9ae0ec705e26d2c9aec6332388d120314730a5c2da183d42c8; SameSite=none; Path=/; Secure; HttpOnly;
-> Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
-> X-Content-Type-Options: nosniff
-> X-Powered-By: ASP.NET
-> nel: {"report_to":"network-errors","max_age":604800,"success_fraction":0.01,"failure_fraction":1.0}
-> report-to: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://mdec.nelreports.net/api/report?cat=mdocs"}]}
-> x-azure-ref: 20251114T141249Z-169b4b8d6588pb5bhC1AMS2m0w00000017n000000000w3q8
BODY Content-Type: text/event-stream
BODY START
event: message
data: {"result":{"content":[{"type":"text","text":"[{\u0022title\u0022:\u0022How to download and install Linux\u0022,\u0022content\u0022:\u0022# How to download and install Linux\\n## Step 2 - Choose a Linux distribution\\nThe different versions of Linux are called \\u0022distributions\\u0022 (sometimes shortened to \\u201Cdistros\\u201D). So there isn\\u2019t actually a way to just \\u201Cinstall Linux\\u201D, you first need to choose which distribution that uses the Linux kernel you want to download and install. There are over 600 active Linux distributions, such as Ubuntu, Debian, Kali Linux, openSUSE, and more. There are a number of factors to consider in choosing a distribution, such as:\\n### Required Linux experience\\nIf you\\u2019re new to Linux, you may want to start with Ubuntu. This is one of the most common starting places for Linux users due to the level of support and ease of use. Ubuntu offers a desktop-version and a server-version depending on how you plan to use it. The specific tasks that the operating system will be responsible for may help you determine which distribution to use. Arch Linux is a popular choice for those who want a highly customizable, do-it-yourself approach that is still stable and well-supported by a large user base. It is a much more complicated place to start, but can help you to get a better understanding of how Linux works due to the amount of custom configuration.\\n### System requirements\\nA minimal, or \\u0022lightweight\\u0022, distribution means a small footprint, with lower memory and processing speed requirements. Alpine Linux is a popular option where users will find most features disabled or not installed by default, but still has everything you need for a containerized app. Security needs: Some Linux users may have specific security concerns to consider. Kali Linux is an example of a distro designed for security tasks, such as Penetration Testing, Security Research, Computer Forensics and Reverse Engineering.\\n### Business and Enterprise requirements\\nBusiness, corporate, or academic environments may have unique concerns from consumer environments. For example Red Hat Enterprise Linux (RHEL), SUSE Linux Enterprise Server (SLES) and Oracle Linux offer 24/7 global support and subscription-based service. CentOS is another popular enterprise distro that is community-supported but still compatible with RHEL.\\n### Community support\u0022,\u0022contentUrl\u0022:\u0022https://learn.microsoft.com/en-us/linux/install#step-2---choose-a-linux-distribution\u0022},{\u0022title\u0022:\u0022New to Linux resources for SQL users\u0022,\u0022content\u0022:\u0022# New 

TextContent(text=[{"title":"How to download and install Linux","content":"# How to download and install Linux\n## Step 2 - Choose a Linux distribution\nThe different versions of Linux are called \u0022distributions\u0022 (sometimes shortened to \u201Cdistros\u201D). So there isn\u2019t actually a way to just \u201Cinstall Linux\u201D, you first need to choose which distribution that uses the Linux kernel you want to download and install. There are over 600 active Linux distributions, such as Ubuntu, Debian, Kali Linux, openSUSE, and more. There are a number of factors to consider in choosing a distribution, such as:\n### Required Linux experience\nIf you\u2019re new to Linux, you may want to start with Ubuntu. This is one of the most common starting places for Linux users due to the level of support and ease of use. Ubuntu offers a desktop-version and a server-version depending on how you plan to use it. The specific tasks that the operating system will be responsible for may help you 